<a href="https://colab.research.google.com/github/SLVmain/NLP/blob/Les_02/HW_02_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Домашка 2. Тема «Создание признакового пространства»

### Задание 1.

**Задание**: обучите три классификатора: 

1) на токенах с высокой частотой 

2) на токенах со средней частотой 

3) на токенах с низкой частотой


Сравните полученные результаты, оцените какие токены наиболее важные для классификации.


### Задание 2.

найти фичи с наибольшей значимостью, и вывести их


### Задание 3.

1) сравнить count/tf-idf/hashing векторайзеры/полносвязанную сетку (построить classification_report)

2) подобрать оптимальный размер для hashing векторайзера 

3) убедиться что для сетки нет переобучения

In [1]:
import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import warnings 
warnings.filterwarnings('ignore')
import os
import time

from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from collections import Counter

import nltk
import html

from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk import tokenize as tknz
from nltk.corpus import stopwords

#nltk.download('genesis')
#nltk.download('punkt')

In [3]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)

## Задача: классификация твитов по тональности

У нас есть датасет из твитов, про каждый указано, как он эмоционально окрашен: положительно или отрицательно. Задача: предсказывать эмоциональную окраску.

Скачиваем куски датасета ([источник](http://study.mokoron.com/)): [положительные](https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0), [отрицательные](https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv).

In [ ]:
#!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
#!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

In [2]:
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = 'positive'
positive.head()

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive


In [3]:
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = 'negative'
df = positive.append(negative)
df.head()

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive


In [6]:
df.describe()

,text,label
count,226834,226834
unique,217440,2
top,"Офигенный день!\nдень позитива)\nбегал как идиот целый день!\nтанцы офигенны, хоть я и ракал но мне очень понравилось!",positive
freq,156,114911


In [4]:
x_train, x_test, y_train, y_test = train_test_split(df.text, df.label)

In [9]:
x_train[:10]

48177     @FenixXx001 все уже, теперь и у меня есть, просто ч ждал пока деньги на карту придут, а дома уже ничего не осталось)
67751                                      Это была контроша по физике...)\nФормулы... \n#школа #физика http://t.co/Eew9lwcwh1
71254                                            @sp_jonny @Dandy_gr @fainarodionovna блин, я прочитала как "неее, Россия!")))
65551                                 А в саноме уже новогоднее настроение:) @ Sanoma Independent Media http://t.co/LSAfoqju5O
61419                                                                             Переезд был тяжелый:( http://t.co/XEapLEluvC
54072                                                                             @sergeykarakich потому что рыбка - еврей((((
103299                                           @MaryVolchik прийдётся вот хоть где то хочу скрыться всё равно не получается(
103108                                                                              @ya_yakusheva я очень скуча

In [10]:
y_train[:10]

48177     positive
67751     positive
71254     positive
65551     positive
61419     negative
54072     negative
103299    negative
103108    negative
36016     positive
111186    negative
Name: label, dtype: object

## Задание 1.

**Задание**: обучите три классификатора: 

1) на токенах с высокой частотой 

2) на токенах со средней частотой 

3) на токенах с низкой частотой

Сравните полученные результаты, оцените какие токены наиболее важные для классификации.

In [11]:
from sklearn.linear_model import LogisticRegression

In [22]:
from string import punctuation
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [23]:
from nltk.corpus import stopwords
nltk.download('stopwords')
my_stopwords = stopwords.words('russian')
noise = stopwords.words('russian') + list(punctuation)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#оставим стоп-слова для полноты картины, чтобы все частоты были представлены

In [55]:
def tfidf_freq_analis(maxdf=1, mindf=1):
    tfidf_vect = TfidfVectorizer(max_df=maxdf, 
                               min_df=mindf,
                             #max_features=1000, #Ограничим количество слов, попадающий в мешок
                             #stop_words=noise, #исключим только знаки препинания
                             ) 
    bow = tfidf_vect.fit_transform(x_train)
    clf = LogisticRegression(random_state=42)
    clf.fit(bow, y_train)
    pred = clf.predict(tfidf_vect.transform(x_test))
    print(f'max_df= {maxdf} min_df= {mindf}')
    print(classification_report(pred, y_test))

In [51]:
params = {'default все токены':[1,1], 'высокая частота': [1.0,0.28], 'средняя частота': [0.7, 0.05], 'низкая частота': [0.4, 1]} # (max, min)

In [52]:
for key, value in params.items():
    print(key)
    tfidf_freq_analis(value[0], value[1])

default все токены
max_df= 1 min_df= 1
              precision    recall  f1-score   support

    negative       0.15      0.58      0.23      7035
    positive       0.90      0.52      0.66     49674

    accuracy                           0.53     56709
   macro avg       0.52      0.55      0.44     56709
weighted avg       0.80      0.53      0.60     56709

высокая частота
max_df= 1.0 min_df= 0.28
              precision    recall  f1-score   support

    negative       0.35      0.60      0.44     16341
    positive       0.77      0.55      0.64     40368

    accuracy                           0.56     56709
   macro avg       0.56      0.58      0.54     56709
weighted avg       0.65      0.56      0.58     56709

средняя частота
max_df= 0.7 min_df= 0.05
              precision    recall  f1-score   support

    negative       0.53      0.59      0.56     25398
    positive       0.63      0.58      0.61     31311

    accuracy                           0.58     56709
   macr

самая низкая точность - если берем высокочастотные токены, самая высокая - когда низкочатотные, что логично, так как низкочастотные токены лучше характеризуют документ

## Задание 2.

найти фичи с наибольшей значимостью, и вывести их

In [63]:
corpus = [token for tweet in df.text for token in word_tokenize(tweet) if token not in punctuation]

In [64]:
counts_ = Counter(corpus)
counts_.most_common(10)

[('не', 69267),
 ('и', 54916),
 ('в', 52853),
 ('я', 52506),
 ('RT', 38070),
 ('на', 35715),
 ('http', 32992),
 ('что', 31472),
 ('...', 28773),
 ('с', 27176)]

In [74]:
freq_dict_sorted= sorted(counts_.items(), key=lambda x: x[1])
list(freq_dict_sorted)[:10]

[('first_timee', 1),
 ('профилирующий', 1),
 ('KatiaCheh', 1),
 ('digger2912', 1),
 ('//t.co/GqG6iuE2…', 1),
 ('irina_dyshkant', 1),
 ('страшилка', 1),
 ('создастся', 1),
 ('//t.co/brf9eNg1U6', 1),
 ('Ох,900', 1)]

In [65]:
corpus_with_punctuations = [token for tweet in df.text for token in word_tokenize(tweet)]

In [67]:
counts_p = Counter(corpus_with_punctuations)
counts_p.most_common(10)

[('(', 212404),
 (')', 194005),
 (',', 188295),
 (':', 177666),
 ('@', 149978),
 ('не', 69267),
 ('!', 66923),
 ('.', 59775),
 ('и', 54916),
 ('в', 52853)]

In [80]:
freq_dict_sorted= sorted(counts_p.items(), key=lambda x: -x[1], reverse=True)
list(freq_dict_sorted)[:10]

[('first_timee', 1),
 ('профилирующий', 1),
 ('KatiaCheh', 1),
 ('digger2912', 1),
 ('//t.co/GqG6iuE2…', 1),
 ('irina_dyshkant', 1),
 ('страшилка', 1),
 ('создастся', 1),
 ('//t.co/brf9eNg1U6', 1),
 ('Ох,900', 1)]

Посмотрим, как один из супер-значительных токенов справится с классификацией безо всякого машинного обучения:

In [83]:
cool_token = ','
pred = ['positive' if cool_token in tweet else 'negative' for tweet in x_test]
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.47      0.50      0.49     26373
    positive       0.54      0.51      0.53     30336

    accuracy                           0.51     56709
   macro avg       0.51      0.51      0.51     56709
weighted avg       0.51      0.51      0.51     56709



In [84]:
cool_token = 'не'
pred = ['positive' if cool_token in tweet else 'negative' for tweet in x_test]
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.47      0.44      0.46     29881
    positive       0.42      0.45      0.43     26828

    accuracy                           0.45     56709
   macro avg       0.45      0.45      0.45     56709
weighted avg       0.45      0.45      0.45     56709



In [85]:
cool_token = '!'
pred = ['positive' if cool_token in tweet else 'negative' for tweet in x_test]
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.87      0.51      0.64     48284
    positive       0.17      0.58      0.26      8425

    accuracy                           0.52     56709
   macro avg       0.52      0.55      0.45     56709
weighted avg       0.77      0.52      0.59     56709



частотные токены сильно увеличивают метрику, так как часто встречаются

### Задание 3.

1) сравнить count/tf-idf/hashing векторайзеры/полносвязанную сетку (построить classification_report)

2) подобрать оптимальный размер для hashing векторайзера 

3) убедиться что для сетки нет переобучения

In [13]:
from sklearn import model_selection, preprocessing, linear_model
from sklearn.metrics import accuracy_score

# labelEncode целевую переменную
encoder = preprocessing.LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)


count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(x_train)

xtrain_count =  count_vect.transform(x_train)
xtest_count =  count_vect.transform(x_test)

classifier = linear_model.LogisticRegression()
classifier.fit(xtrain_count, y_train)
predictions = classifier.predict(xtest_count)
#predictions
print(accuracy_score(y_test, predictions))
print(classification_report(predictions, y_test))

0.7839496376236577
              precision    recall  f1-score   support

           0       0.79      0.78      0.78     28678
           1       0.78      0.79      0.78     28031

    accuracy                           0.78     56709
   macro avg       0.78      0.78      0.78     56709
weighted avg       0.78      0.78      0.78     56709



In [28]:
tf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_df=0.3)
tf_vect.fit(x_train)

xtrain_count =  tf_vect.transform(x_train)
xtest_count =  tf_vect.transform(x_test)

classifier = linear_model.LogisticRegression()
classifier.fit(xtrain_count, y_train)
predictions = classifier.predict(xtest_count)
#predictions
print(accuracy_score(y_test, predictions))
print(classification_report(predictions, y_test))

0.7815690631116754
              precision    recall  f1-score   support

           0       0.78      0.78      0.78     28161
           1       0.78      0.78      0.78     28548

    accuracy                           0.78     56709
   macro avg       0.78      0.78      0.78     56709
weighted avg       0.78      0.78      0.78     56709



In [20]:
from sklearn.feature_extraction.text import HashingVectorizer
list_ = [200,2000, 20000, 200000, 2000000]
for i in list_:
    h_vect = HashingVectorizer(n_features=i)
    h_vect.fit(x_train)

    xtrain_count =  h_vect.transform(x_train)
    xtest_count =  h_vect.transform(x_test)

    classifier = linear_model.LogisticRegression()
    classifier.fit(xtrain_count, y_train)
    predictions = classifier.predict(xtest_count)
    #predictions
    print(f'размер вектора hash= {i}')
    print(accuracy_score(y_test, predictions))
    print(classification_report(predictions, y_test))

размер вектора hash= 200
0.611155195824296
              precision    recall  f1-score   support

           0       0.60      0.61      0.60     27431
           1       0.63      0.61      0.62     29278

    accuracy                           0.61     56709
   macro avg       0.61      0.61      0.61     56709
weighted avg       0.61      0.61      0.61     56709

размер вектора hash= 2000
0.6682184485707736
              precision    recall  f1-score   support

           0       0.65      0.67      0.66     27301
           1       0.68      0.67      0.68     29408

    accuracy                           0.67     56709
   macro avg       0.67      0.67      0.67     56709
weighted avg       0.67      0.67      0.67     56709

размер вектора hash= 20000
0.7266571443686187
              precision    recall  f1-score   support

           0       0.71      0.73      0.72     27223
           1       0.74      0.72      0.73     29486

    accuracy                           0.73     

даже с 2 млн хэш дает метрики ниже обычного и tfidf векторайзера, можно остановиться на 20000, в зависимости от ресурсов.

Модели не переобучены, так как для валидации использовала тестовую выборку

кусок старой домашки, так как сначала ее начала делать

In [ ]:
my_path = '/content/drive/MyDrive/GB_NLP_04_2022/01/Homework01/'

In [ ]:
#загружаем таблицу, уже сделан препроцессинг в домашке 1
df = pd.read_pickle(my_path + 'df.pkl')

In [ ]:
df.head()

,id,label,tweet,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when father is dysfunctional and is so selfish he drags his kids into his dysfunction run,"[when, father, is, dysfunctional, and, is, so, selfish, he, drags, his, kids, into, his, dysfunction, run]","[father, dysfunctional, selfish, drags, kids, dysfunction, run]","[father, dysfunct, selfish, drag, kid, dysfunct, run]","[father, dysfunctional, selfish, drag, kid, dysfunction, run]"
1,2,0.0,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,thanks for lyft credit can use cause they don offer wheelchair vans in pdx disapointed getthanked,"[thanks, for, lyft, credit, can, use, cause, they, don, offer, wheelchair, vans, in, pdx, disapointed, getthanked]","[thanks, lyft, credit, use, cause, offer, wheelchair, vans, pdx, disapointed, getthanked]","[thank, lyft, credit, use, caus, offer, wheelchair, van, pdx, disapoint, getthank]","[thank, lyft, credit, use, cause, offer, wheelchair, vans, pdx, disapointed, getthanked]"
2,3,0.0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]","[bihday, majesty]"
3,4,0.0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦,model love take with all the time in ur,"[model, love, take, with, all, the, time, in, ur]","[model, love, take, time, ur]","[model, love, take, time, ur]","[model, love, take, time, ur]"
4,5,0.0,factsguide: society now #motivation,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, motiv]","[factsguide, society, motivation]"


In [ ]:
# сделаем цельный текст из списков
for col in ['tweet_stemmed', 'tweet_lemmatized']:
    df[col] = df[col].apply(lambda list_element: ' '.join(list_element))

df.head()

,id,label,tweet,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when father is dysfunctional and is so selfish he drags his kids into his dysfunction run,"[when, father, is, dysfunctional, and, is, so, selfish, he, drags, his, kids, into, his, dysfunction, run]","[father, dysfunctional, selfish, drags, kids, dysfunction, run]",father dysfunct selfish drag kid dysfunct run,father dysfunctional selfish drag kid dysfunction run
1,2,0.0,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,thanks for lyft credit can use cause they don offer wheelchair vans in pdx disapointed getthanked,"[thanks, for, lyft, credit, can, use, cause, they, don, offer, wheelchair, vans, in, pdx, disapointed, getthanked]","[thanks, lyft, credit, use, cause, offer, wheelchair, vans, pdx, disapointed, getthanked]",thank lyft credit use caus offer wheelchair van pdx disapoint getthank,thank lyft credit use cause offer wheelchair vans pdx disapointed getthanked
2,3,0.0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]",bihday majesti,bihday majesty
3,4,0.0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦,model love take with all the time in ur,"[model, love, take, with, all, the, time, in, ur]","[model, love, take, time, ur]",model love take time ur,model love take time ur
4,5,0.0,factsguide: society now #motivation,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]",factsguid societi motiv,factsguide society motivation


1. Создайте мешок слов с помощью sklearn.feature_extraction.text.CountVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.
Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.
Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.
Исключим стоп-слова с помощью stop_words='english'. 
Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью CountVectorizer.get_feature_names().

In [ ]:
count_vect_stem = CountVectorizer(max_df=0.9, #Игнорируем слова, частота которых в документе строго превышает порог 0.9
                             max_features=1000, #Ограничим количество слов, попадающий в мешок
                             stop_words='english', #исключим стоп-слова
                             ) 
count_vect_lem = CountVectorizer(max_df=0.9, #Игнорируем слова, частота которых в документе строго превышает порог 0.9
                             max_features=1000, #Ограничим количество слов, попадающий в мешок
                             stop_words='english', #исключим стоп-слова
                             ) 

In [ ]:
bow_tweet_stem = count_vect_stem.fit_transform(df['tweet_stemmed'])
bow_tweet_lem = count_vect_lem.fit_transform(df['tweet_lemmatized'])

In [ ]:
columns_stem = count_vect_stem.get_feature_names()
columns_lem = count_vect_lem.get_feature_names()

In [ ]:
bow_stem_df = pd.DataFrame(bow_tweet_stem.toarray(), columns=columns_stem)

In [ ]:
bow_stem_df.head(2)

,abl,absolut,accept,account,act,action,actor,actual,ad,adapt,add,adventur,affirm,afternoon,age,ago,agre,ahead,aist,album,aliv,allahsoil,allow,alon,alreadi,altwaystoh,alway,amaz,america,american,angel,anger,angri,anim,anniversari,announc,anoth,answer,anti,anxieti,anymor,anyon,anyth,app,appl,appreci,arriv,ask,ass,attack,august,avail,award,away,awesom,aww,babi,bad,bag,balanc,ball,bar,bc,bday,beach,bear,beat,beauti,becom,bed,beer,begin,believ,benefit,best,bestfriend,besti,better,bff,big,bigot,bihday,bike,bing,bird,bit,bitch,black,blame,bless,blm,block,blog,blogger,blond,blue,blur,board,bodi,bong,book,bore,bought,box,boy,boyfriend,brand,break,breakfast,brexit,bride,bring,bro,broken,brother,brown,buffalo,build,bull,busi,buy,cake,calm,came,camp,campaign,cantwait,car,card,care,case,cat,caus,celebr,challeng,chanc,chang,chase,check,cheer,child,children,chill,choic,choos,christian,christina,christma,church,citi,claim,class,clean,client,climb,close,cloth,club,coach,coffe,cold,collect,colleg,color,colour,come,comment,commun,compani,complet,conc,confer,confid,confus,congrat,congratul,connect,content,continu,control,cook,cool,count,countdown,countri,coupl,cours,cover,crazi,creat,creativ,cri,cultur,cup,current,custom,cut,cute,da,dad,daddi,daili,damn,danc,dark,date,daughter,day,dead,deal,dear,death,decid,deep,definit,delet,deletetweet,depress,deserv,design,despit,develop,die,diet,differ,dinner,direct,disappoint,disgust,disney,dj,dog,domin,donald,dont,draw,dream,dress,drink,drive,drop,dude,eah,earli,easi,eat,edit,educ,elect,els,em,email,emot,empti,end,energi,england,enjoy,environ,episod,equal,especi,eur,euro,event,everi,everybodi,everyday,everyon,everyth,ex,exactli,exam,excit,expans,expect,experi,eye,face,facebook,fact,fail,faith,fake,fall,famili,fan,fantast,far,fashion,fast,father,fathersday,favorit,favourit,fear,feel,femal,festiv,ff,fight,film,final,finger,finish,fit,flag,fli,flight,florida,flower,folk,follow,followm,food,foodi,footbal,forev,forex,forget,forward,franc,free,freedom,fresh,friday,fridayfeel,friend,friendship,fuck,fun,funni,futur,game,garden,gave,gay,gbp,gener,germani,gif,gift,girl,glad,glass,goal,god,goe,gold,gon,gone,good,goodmorn,goodnight,goodtim,goodvib,gop,gorgeou,gorilla,got,grace,graduat,grate,gratitud,great,green,group,grow,guess,gun,guy,gym,haha,hair,half,hand,happen,happi,happier,hard,hardcor,hate,hatr,hea,heabreak,heabroken,head,heal,health,healthi,hear,heard,hell,hello,help,hero,hey,hi,high,hill,hillari,histori,hit,hold,holiday,home,hope,horni,horribl,hot,hotel,hour,hous,hr,ht,htt,hu,hug,huge,human,hungri,husband,ibiza,ice,idea,idiot,ignor,im,imagin,impoant,impress,india,inshot,insid,inspir,instadaili,instagood,instagram,instalik,instamood,instead,intern,internet,islam,issu,japan,job,join,joke,journey,joy,juli,june,key,kick,kid,kill,kind,king,kinki,kiss,knew,know,la,ladi,land,late,later,latest,laugh,launch,law,lawofattract,lead,leader,leadership,leakag,learn,leav,left,legend,let,level,lgbt,liber,libtard,lie,life,lifestyl,light,lighttherapi,like,line,link,list,listen,liter,littl,live,local,lol,london,lone,long,look,lose,loser,loss,lost,lot,love,loveit,lover,low,luck,lucki,lunch,mad,magic,make,makeup,man,manag,mani,market,marri,marriag,mass,match,materi,matter,mayb,mean,media,meet,melancholi,member,meme,memori,men,messag,met,miami,million,min,mind,mindset,minut,miss,model,mom,moment,monday,money,month,mood,morn,mother,motiv,mountain,movi,mr,murder,music,muslim,na,nail,nake,nasti,nation,natur,naughti,nba,nbafin,nd,need,nervou,new,news,newyork,nice,nigga,night,nobodi,nofilt,noth,notic,nude,number,nyc,obama,offer,offic,offici,oh,oil,oitnb,ok,okay,old,omg,onlin,ootd,open,orang,order,organ,origin,orlando,orlandoshoot,outsid,pa,pack,page,pain,paint,parent,pari,park,pass,passion,past,pathet,pay,peac,peopl,perfect,perform,person,pet,phone,photo,photographi,photooftheday,pic,pick,picoftheday,pictur,pig,pink,pizza,place,plan,play,player,pleas,pm,poetri,point,polar,polic,polit,politician,pool,poor,pop,porn,posit,possibl,post,powe

In [ ]:
bow_lem_df = pd.DataFrame(bow_tweet_lem.toarray(), columns=columns_lem)
bow_lem_df.head(2)

,able,absolutely,accept,account,act,action,actor,actually,adapt,add,adventure,affirmation,affirmations,afternoon,age,ago,agree,ahead,aist,album,alive,allahsoil,alligator,allow,altwaystoheal,amaze,america,american,americans,anger,angry,animals,anniversary,announce,answer,anti,anxiety,anymore,app,apple,appreciate,arrive,ask,ass,attack,august,available,award,away,awesome,aww,baby,bad,bag,balance,ball,bank,bar,bc,bday,beach,bear,beat,beautiful,beauty,bed,beer,begin,behappy,believe,benefit,best,better,big,bihday,bike,bing,bird,bitch,bite,black,blame,bless,blm,block,blog,blogger,blonde,blue,blur,board,body,bong,book,bore,box,boy,boyfriend,boys,brand,break,breakfast,brexit,bride,bring,bro,brother,brown,buffalo,build,bull,business,busy,buy,cake,calm,camp,campaign,cantwait,car,card,care,case,cat,catch,cause,cavs,celebrate,celebration,challenge,chance,change,chase,check,cheer,child,children,chill,chocolate,choice,choose,christina,christmas,church,city,claim,class,clean,click,climb,close,clothe,club,coach,cock,coffee,cold,college,color,come,comedy,comment,community,company,complete,conce,conference,confuse,congrats,congratulations,content,continue,control,cook,cool,count,countdown,country,couple,course,cover,crazy,create,creative,culture,cup,customer,customers,cut,cute,da,dad,daddy,dads,daily,damn,dance,dark,date,daughter,day,days,dead,deal,dear,death,decide,deep,delete,deletetweets,delicious,depress,depression,deserve,design,despite,development,die,diet,different,dinner,direct,disappoint,disgust,disney,dj,dog,dominate,donald,dont,download,draw,dream,dress,drink,drive,drop,eah,early,easy,eat,education,election,em,email,emotional,emotions,end,energy,england,enjoy,environment,episode,especially,essentialoils,eur,euro,euros,event,events,everybody,everyday,evil,ex,exactly,excite,exercise,expanse,expect,experience,eye,face,facebook,fact,fail,faith,fake,fall,families,family,fan,fantastic,far,fashion,fast,father,fathersday,favorite,favourite,fear,feel,female,festival,ff,fight,film,final,finally,finger,finish,fit,fitness,flag,flight,florida,flower,fly,focus,folks,follow,followers,followme,food,football,force,forever,forex,forget,forward,france,free,freedom,fresh,friday,fridayfeeling,friend,friends,friendship,fuck,fun,funny,future,game,garden,gay,gbp,germany,gif,gift,girl,girls,glad,glass,goals,god,gold,gon,good,goodmorning,goodnight,goodtimes,goodvibes,gop,gorgeous,gorilla,graduation,grateful,gratitude,great,green,group,grow,guess,gun,guy,gym,haha,hair,half,hand,hang,happen,happier,happiness,happy,hard,hardcore,hat,hate,hatred,hea,heabroken,head,heal,health,healthy,hear,hell,hello,help,hero,hey,hi,high,hill,hillary,history,hit,hold,holiday,home,hop,hope,horny,horrible,hot,hotel,hour,hours,house,hrs,ht,htt,hu,hug,huge,human,humpday,hungry,husband,ibiza,ice,idea,ignorance,ignore,im,imagine,impoant,india,inshot,inside,inspiration,inspirational,inspire,instadaily,instagood,instagram,instalike,instamood,instead,internet,islam,issue,japan,job,join,joke,journey,joy,july,june,key,kick,kid,kill,kind,king,kinky,kiss,know,la,lack,ladies,lady,land,late,later,latest,laugh,launch,law,lawofattraction,lead,leadership,leakage,learn,leave,let,level,lgbt,liberal,libtard,lie,life,lifestyle,light,lighttherapy,like,line,link,list,listen,literally,little,live,local,lol,london,lonely,long,longer,look,lose,loss,lot,love,loveit,lovelife,lovely,lover,low,luck,lucky,lunch,lyric,mad,magic,make,makeup,man,market,marriage,marry,mass,match,matter,maybe,mean,media,meet,melancholy,memories,men,message,miami,mind,mindset,minute,minutes,miss,model,mom,moment,moments,monday,money,month,months,mood,morning,mother,motivate,motivation,mountains,movie,movies,mr,murder,music,muslim,muslims,na,nail,naked,nasty,national,nature,naughty,nba,nbafinals,nd,near,need,nervous,ness,netflix,new,news,newyork,nice,night,nofilter,notice,nude,number,nyc,obama,offer,office,official,officially,oh,oil,oitnb,ok,okay,old,omg,ones,online,ootd,open,orange,order,organizations,orlando,orlandoshooting,outside,p

2. Создайте мешок слов с помощью sklearn.feature_extraction.text.TfidfVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.
Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.
Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.
Исключим стоп-слова с помощью stop_words='english'.
Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью TfidfVectorizer.get_feature_names().

In [ ]:
tf_vect_stem = TfidfVectorizer(max_df=0.9, #Игнорируем слова, частота которых в документе строго превышает порог 0.9
                             max_features=1000, #Ограничим количество слов, попадающий в мешок
                             stop_words='english', #исключим стоп-слова
                             ) 
tf_vect_lem = TfidfVectorizer(max_df=0.9, #Игнорируем слова, частота которых в документе строго превышает порог 0.9
                             max_features=1000, #Ограничим количество слов, попадающий в мешок
                             stop_words='english', #исключим стоп-слова
                             ) 
tf_tweet_stem = tf_vect_stem.fit_transform(df['tweet_stemmed'])
tf_tweet_lem = tf_vect_lem.fit_transform(df['tweet_lemmatized'])

tf_columns_stem = tf_vect_stem.get_feature_names()
tf_columns_lem = tf_vect_lem.get_feature_names()

In [ ]:
tf_stem_df = pd.DataFrame(tf_tweet_stem.toarray(), columns=tf_columns_stem)
tf_stem_df.head(2)

,abl,absolut,accept,account,act,action,actor,actual,ad,adapt,add,adventur,affirm,afternoon,age,ago,agre,ahead,aist,album,aliv,allahsoil,allow,alon,alreadi,altwaystoh,alway,amaz,america,american,angel,anger,angri,anim,anniversari,announc,anoth,answer,anti,anxieti,anymor,anyon,anyth,app,appl,appreci,arriv,ask,ass,attack,august,avail,award,away,awesom,aww,babi,bad,bag,balanc,ball,bar,bc,bday,beach,bear,beat,beauti,becom,bed,beer,begin,believ,benefit,best,bestfriend,besti,better,bff,big,bigot,bihday,bike,bing,bird,bit,bitch,black,blame,bless,blm,block,blog,blogger,blond,blue,blur,board,bodi,bong,book,bore,bought,box,boy,boyfriend,brand,break,breakfast,brexit,bride,bring,bro,broken,brother,brown,buffalo,build,bull,busi,buy,cake,calm,came,camp,campaign,cantwait,car,card,care,case,cat,caus,celebr,challeng,chanc,chang,chase,check,cheer,child,children,chill,choic,choos,christian,christina,christma,church,citi,claim,class,clean,client,climb,close,cloth,club,coach,coffe,cold,collect,colleg,color,colour,come,comment,commun,compani,complet,conc,confer,confid,confus,congrat,congratul,connect,content,continu,control,cook,cool,count,countdown,countri,coupl,cours,cover,crazi,creat,creativ,cri,cultur,cup,current,custom,cut,cute,da,dad,daddi,daili,damn,danc,dark,date,daughter,day,dead,deal,dear,death,decid,deep,definit,delet,deletetweet,depress,deserv,design,despit,develop,die,diet,differ,dinner,direct,disappoint,disgust,disney,dj,dog,domin,donald,dont,draw,dream,dress,drink,drive,drop,dude,eah,earli,easi,eat,edit,educ,elect,els,em,email,emot,empti,end,energi,england,enjoy,environ,episod,equal,especi,eur,euro,event,everi,everybodi,everyday,everyon,everyth,ex,exactli,exam,excit,expans,expect,experi,eye,face,facebook,fact,fail,faith,fake,fall,famili,fan,fantast,far,fashion,fast,father,fathersday,favorit,favourit,fear,feel,femal,festiv,ff,fight,film,final,finger,finish,fit,flag,fli,flight,florida,flower,folk,follow,followm,food,foodi,footbal,forev,forex,forget,forward,franc,free,freedom,fresh,friday,fridayfeel,friend,friendship,fuck,fun,funni,futur,game,garden,gave,gay,gbp,gener,germani,gif,gift,girl,glad,glass,goal,god,goe,gold,gon,gone,good,goodmorn,goodnight,goodtim,goodvib,gop,gorgeou,gorilla,got,grace,graduat,grate,gratitud,great,green,group,grow,guess,gun,guy,gym,haha,hair,half,hand,happen,happi,happier,hard,hardcor,hate,hatr,hea,heabreak,heabroken,head,heal,health,healthi,hear,heard,hell,hello,help,hero,hey,hi,high,hill,hillari,histori,hit,hold,holiday,home,hope,horni,horribl,hot,hotel,hour,hous,hr,ht,htt,hu,hug,huge,human,hungri,husband,ibiza,ice,idea,idiot,ignor,im,imagin,impoant,impress,india,inshot,insid,inspir,instadaili,instagood,instagram,instalik,instamood,instead,intern,internet,islam,issu,japan,job,join,joke,journey,joy,juli,june,key,kick,kid,kill,kind,king,kinki,kiss,knew,know,la,ladi,land,late,later,latest,laugh,launch,law,lawofattract,lead,leader,leadership,leakag,learn,leav,left,legend,let,level,lgbt,liber,libtard,lie,life,lifestyl,light,lighttherapi,like,line,link,list,listen,liter,littl,live,local,lol,london,lone,long,look,lose,loser,loss,lost,lot,love,loveit,lover,low,luck,lucki,lunch,mad,magic,make,makeup,man,manag,mani,market,marri,marriag,mass,match,materi,matter,mayb,mean,media,meet,melancholi,member,meme,memori,men,messag,met,miami,million,min,mind,mindset,minut,miss,model,mom,moment,monday,money,month,mood,morn,mother,motiv,mountain,movi,mr,murder,music,muslim,na,nail,nake,nasti,nation,natur,naughti,nba,nbafin,nd,need,nervou,new,news,newyork,nice,nigga,night,nobodi,nofilt,noth,notic,nude,number,nyc,obama,offer,offic,offici,oh,oil,oitnb,ok,okay,old,omg,onlin,ootd,open,orang,order,organ,origin,orlando,orlandoshoot,outsid,pa,pack,page,pain,paint,parent,pari,park,pass,passion,past,pathet,pay,peac,peopl,perfect,perform,person,pet,phone,photo,photographi,photooftheday,pic,pick,picoftheday,pictur,pig,pink,pizza,place,plan,play,player,pleas,pm,poetri,point,polar,polic,polit,politician,pool,poor,pop,porn,posit,possibl,post,powe

In [ ]:
tf_lem_df = pd.DataFrame(tf_tweet_lem.toarray(), columns=tf_columns_lem)
tf_lem_df.head(2)

,able,absolutely,accept,account,act,action,actor,actually,adapt,add,adventure,affirmation,affirmations,afternoon,age,ago,agree,ahead,aist,album,alive,allahsoil,alligator,allow,altwaystoheal,amaze,america,american,americans,anger,angry,animals,anniversary,announce,answer,anti,anxiety,anymore,app,apple,appreciate,arrive,ask,ass,attack,august,available,award,away,awesome,aww,baby,bad,bag,balance,ball,bank,bar,bc,bday,beach,bear,beat,beautiful,beauty,bed,beer,begin,behappy,believe,benefit,best,better,big,bihday,bike,bing,bird,bitch,bite,black,blame,bless,blm,block,blog,blogger,blonde,blue,blur,board,body,bong,book,bore,box,boy,boyfriend,boys,brand,break,breakfast,brexit,bride,bring,bro,brother,brown,buffalo,build,bull,business,busy,buy,cake,calm,camp,campaign,cantwait,car,card,care,case,cat,catch,cause,cavs,celebrate,celebration,challenge,chance,change,chase,check,cheer,child,children,chill,chocolate,choice,choose,christina,christmas,church,city,claim,class,clean,click,climb,close,clothe,club,coach,cock,coffee,cold,college,color,come,comedy,comment,community,company,complete,conce,conference,confuse,congrats,congratulations,content,continue,control,cook,cool,count,countdown,country,couple,course,cover,crazy,create,creative,culture,cup,customer,customers,cut,cute,da,dad,daddy,dads,daily,damn,dance,dark,date,daughter,day,days,dead,deal,dear,death,decide,deep,delete,deletetweets,delicious,depress,depression,deserve,design,despite,development,die,diet,different,dinner,direct,disappoint,disgust,disney,dj,dog,dominate,donald,dont,download,draw,dream,dress,drink,drive,drop,eah,early,easy,eat,education,election,em,email,emotional,emotions,end,energy,england,enjoy,environment,episode,especially,essentialoils,eur,euro,euros,event,events,everybody,everyday,evil,ex,exactly,excite,exercise,expanse,expect,experience,eye,face,facebook,fact,fail,faith,fake,fall,families,family,fan,fantastic,far,fashion,fast,father,fathersday,favorite,favourite,fear,feel,female,festival,ff,fight,film,final,finally,finger,finish,fit,fitness,flag,flight,florida,flower,fly,focus,folks,follow,followers,followme,food,football,force,forever,forex,forget,forward,france,free,freedom,fresh,friday,fridayfeeling,friend,friends,friendship,fuck,fun,funny,future,game,garden,gay,gbp,germany,gif,gift,girl,girls,glad,glass,goals,god,gold,gon,good,goodmorning,goodnight,goodtimes,goodvibes,gop,gorgeous,gorilla,graduation,grateful,gratitude,great,green,group,grow,guess,gun,guy,gym,haha,hair,half,hand,hang,happen,happier,happiness,happy,hard,hardcore,hat,hate,hatred,hea,heabroken,head,heal,health,healthy,hear,hell,hello,help,hero,hey,hi,high,hill,hillary,history,hit,hold,holiday,home,hop,hope,horny,horrible,hot,hotel,hour,hours,house,hrs,ht,htt,hu,hug,huge,human,humpday,hungry,husband,ibiza,ice,idea,ignorance,ignore,im,imagine,impoant,india,inshot,inside,inspiration,inspirational,inspire,instadaily,instagood,instagram,instalike,instamood,instead,internet,islam,issue,japan,job,join,joke,journey,joy,july,june,key,kick,kid,kill,kind,king,kinky,kiss,know,la,lack,ladies,lady,land,late,later,latest,laugh,launch,law,lawofattraction,lead,leadership,leakage,learn,leave,let,level,lgbt,liberal,libtard,lie,life,lifestyle,light,lighttherapy,like,line,link,list,listen,literally,little,live,local,lol,london,lonely,long,longer,look,lose,loss,lot,love,loveit,lovelife,lovely,lover,low,luck,lucky,lunch,lyric,mad,magic,make,makeup,man,market,marriage,marry,mass,match,matter,maybe,mean,media,meet,melancholy,memories,men,message,miami,mind,mindset,minute,minutes,miss,model,mom,moment,moments,monday,money,month,months,mood,morning,mother,motivate,motivation,mountains,movie,movies,mr,murder,music,muslim,muslims,na,nail,naked,nasty,national,nature,naughty,nba,nbafinals,nd,near,need,nervous,ness,netflix,new,news,newyork,nice,night,nofilter,notice,nude,number,nyc,obama,offer,office,official,officially,oh,oil,oitnb,ok,okay,old,omg,ones,online,ootd,open,orange,order,organizations,orlando,orlandoshooting,outside,p